In [1]:
import pandas as pd
import numpy as np
import hashlib
import string
import random

In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [3]:
df = pd.read_csv('data/Kobizo AI engineer test - dataset sample - Sheet1.csv')
df

,transaction id,time stamp,from,to,value,method called,token price,liquidity,market cap
0,0xf04403e162a8a988780de9a0416f8b87633e71435f03...,Jul-30-2024 01:53:37 PM UTC,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x00000000041d945c46E073F0048cEf510D148dEA,$0.12,buy,$2.38,"$1,153,212.96","$3,234,533.45"
1,0x5fb4348e6ef84249f12ec5058e373e956d95913dcdec...,Jul-30-2024 01:53:36 PM UTC,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x00000000041d945c46E073F0048cEf510D148dEA,$0.02,buy,$2.39,"$1,153,234.56","$3,235,002.04"
2,0x1bf427e14853d650e332b6188962e1ea3b21c02492e2...,Jul-30-2024 12:34:38 AM UTC,0xdA07eFFA7aEe8CD85feb894e33Da9163f082D19e,0x46B3fDF7b5CDe91Ac049936bF0bDb12c5d22202e,$2.40,buy,$2.40,"$1,153,266.32","$3,234,113.31"
3,0x737bb2b517312cd2166bacb41daf5c1a80e560d3f0c8...,Jul-30-2024 12:02:23 AM UTC,0x639D2a49bdD0CF37614F46a83e4AA49eAFF1e537,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,$5,transfer,$2.39,"$1,153,265.16","$3,234,323.80"
4,0x94002f1eabc5b491f9f143bd761ff693362c25e2832f...,Jul-29-2024 21:05:16 AM UTC,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,$3.50,swap,$2.26,"$1,153,344.33","$3,235,421.46"
5,0xe4ecd37edb93d22eef32f62dcb397f23f75b699a0c53...,Jul-29-2024 21:05:15 AM UTC,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,$3.50,swap,$2.26,"$1,153,342.55","$3,235,532.40"
6,0x0232c4dd9f4b88222ed95a25f6236d7dc25b9d3dccb9...,Jul-29-2024 21:05:14 AM UTC,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,$3.50,swap,$2.22,"$1,153,312.90","$3,236,754.55"
7,0xc5061da6f7c89ab0fc20c912f058636a8c3f9d8b0456...,Jul-29-2024 21:05:13 AM UTC,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,$3.50,swap,$2.20,"$1,153,319.12","$3,236,751.09"
8,0x34484bf7d85bc584599957718112f0dc309a45208457...,Jul-30-2024 01:53:38 PM UTC,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x00000000041d945c46E073F0048cEf510D148dEA,"$2,110",printMoney,$2.21,"$1,153,278.06","$3,235,980.11"


In [4]:
df.dtypes

transaction id    object
time stamp        object
from              object
to                object
value             object
method called     object
token price       object
liquidity         object
market cap        object
dtype: object

In [5]:
df['time stamp'] = df['time stamp'].str.replace(r'\s+(AM|PM)', '', regex=True) # inconsitant timestamp
df['time stamp'] = pd.to_datetime(df['time stamp'], format='%b-%d-%Y %H:%M:%S %Z')

In [6]:
# convert currency column to float
convert_col = ['value', 'token price', 'liquidity', 'market cap']
for c in convert_col:
    df[c] = df[c].str.replace('$', '').str.replace(',', '').astype(float)
df

,transaction id,time stamp,from,to,value,method called,token price,liquidity,market cap
0,0xf04403e162a8a988780de9a0416f8b87633e71435f03...,2024-07-30 01:53:37+00:00,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x00000000041d945c46E073F0048cEf510D148dEA,0.12,buy,2.38,1153212.96,3234533.45
1,0x5fb4348e6ef84249f12ec5058e373e956d95913dcdec...,2024-07-30 01:53:36+00:00,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x00000000041d945c46E073F0048cEf510D148dEA,0.02,buy,2.39,1153234.56,3235002.04
2,0x1bf427e14853d650e332b6188962e1ea3b21c02492e2...,2024-07-30 12:34:38+00:00,0xdA07eFFA7aEe8CD85feb894e33Da9163f082D19e,0x46B3fDF7b5CDe91Ac049936bF0bDb12c5d22202e,2.40,buy,2.40,1153266.32,3234113.31
3,0x737bb2b517312cd2166bacb41daf5c1a80e560d3f0c8...,2024-07-30 12:02:23+00:00,0x639D2a49bdD0CF37614F46a83e4AA49eAFF1e537,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,5.00,transfer,2.39,1153265.16,3234323.80
4,0x94002f1eabc5b491f9f143bd761ff693362c25e2832f...,2024-07-29 21:05:16+00:00,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,3.50,swap,2.26,1153344.33,3235421.46
5,0xe4ecd37edb93d22eef32f62dcb397f23f75b699a0c53...,2024-07-29 21:05:15+00:00,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,3.50,swap,2.26,1153342.55,3235532.40
6,0x0232c4dd9f4b88222ed95a25f6236d7dc25b9d3dccb9...,2024-07-29 21:05:14+00:00,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,3.50,swap,2.22,1153312.90,3236754.55
7,0xc5061da6f7c89ab0fc20c912f058636a8c3f9d8b0456...,2024-07-29 21:05:13+00:00,0x00000000fc65059bDdA566b43E17c310B3076e33,0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf,3.50,swap,2.20,1153319.12,3236751.09
8,0x34484bf7d85bc584599957718112f0dc309a45208457...,2024-07-30 01:53:38+00:00,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x00000000041d945c46E073F0048cEf510D148dEA,2110.00,printMoney,2.21,1153278.06,3235980.11


In [7]:
df['to'].value_counts()

0x4E3288c9ca110bCC82bf38F09A7b425c095d92Bf    5
0x00000000041d945c46E073F0048cEf510D148dEA    3
0x46B3fDF7b5CDe91Ac049936bF0bDb12c5d22202e    1
Name: to, dtype: int64

In [8]:
df['time stamp'].min()

Timestamp('2024-07-29 21:05:13+0000', tz='UTC')

In [9]:
df['liquidity'].describe()

count         9.00
mean    1153286.22
std          46.48
min     1153212.96
25%     1153265.16
50%     1153278.06
75%     1153319.12
max     1153344.33
Name: liquidity, dtype: float64

In [10]:
# generate sample

In [11]:
def generate_list_with_outliers(min_value, max_value, num_samples=100, outlier_perc=0.01):
    # Generate normal values within the range
    normal_values = [random.uniform(min_value, max_value) for _ in range(int(num_samples*(1-outlier_perc)))]
    # Generate outliers outside the range
    outlier_values = [random.uniform(max_value * 10, max_value * 20) for _ in range(int(num_samples*outlier_perc))]
    # Combine and shuffle the list
    full_list = normal_values + outlier_values
    random.shuffle(full_list)
    return full_list
#
def generate_timestamps(min_date, max_date, num_samples=100):
    # Convert min_date and max_date strings to pandas Timestamps
    min_date = pd.to_datetime(min_date)
    max_date = pd.to_datetime(max_date)
    # Generate evenly spaced timestamps between min_date and max_date
    timestamps = pd.date_range(start=min_date, end=max_date, periods=num_samples)
    # Convert to a list of strings
    return timestamps.strftime('%Y-%m-%d %H:%M:%S').tolist()
#
def generate_standard_normal_distribution(min_value, max_value, num_values=100):
    # Generate values from standard normal distribution (mean=0, std=1)
    values = np.random.normal(loc=0, scale=1, size=num_values)
    # Scale the values to the range [min_value, max_value]
    scaled_values = min_value + (values - values.min()) * (max_value - min_value) / (values.max() - values.min())
    return scaled_values.tolist()
#
def generate_hash_id(length=66):
    # Generate a random string (e.g., 32 characters long)
    random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=32))
    # Hash the random string using SHA-256
    hash_object = hashlib.sha256(random_string.encode())
    hash_hex = hash_object.hexdigest()
    # Return the hash, truncated or padded to the desired length (66 characters)
    return hash_hex[:length].ljust(length, '0')  # Pad with '0' if needed
#
def generate_hash_list(num_values=100, length=66):
    return [generate_hash_id(length) for _ in range(num_values)]

In [12]:
n_sample = 10000
txn_id = generate_hash_list(num_values=n_sample)
time_stamp = generate_timestamps('2024-07-29', '2024-08-02', num_samples=n_sample)
txn_maker_address = random.choices(list(set(df['from'].tolist() + df['to'].tolist())), k=n_sample)
txn_receiver_address = random.choices(list(set(df['from'].tolist() + df['to'].tolist())), k=n_sample)
txn_value = generate_list_with_outliers(2, 4, num_samples=n_sample, outlier_perc=0.01)
method_called = random.choices(df['method called'].unique(), k=n_sample)
token_price = generate_standard_normal_distribution(df['token price'].min(), df['token price'].max(), num_values=n_sample)
liquidity =  generate_standard_normal_distribution(df['liquidity'].min(), df['liquidity'].max(), num_values=n_sample)
market_cap =  generate_standard_normal_distribution(df['market cap'].min(), df['market cap'].max(), num_values=n_sample)

In [13]:
generate_df = pd.DataFrame({
    'txn_id': txn_id,
    'time_stamp': time_stamp,
    'txn_maker_address': txn_maker_address,
    'txn_receiver_address': txn_receiver_address,
    'txn_value': txn_value,
    'method_called': method_called,
    'token_price': token_price,
    'liquidity': liquidity,
    'market_cap': market_cap,
})
generate_df

,txn_id,time_stamp,txn_maker_address,txn_receiver_address,txn_value,method_called,token_price,liquidity,market_cap
0,85214fe243b6d9be9765c12ed0ed32644464437e218eaf...,2024-07-29 00:00:00,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x46B3fDF7b5CDe91Ac049936bF0bDb12c5d22202e,2.19,swap,2.30,1153266.06,3235211.29
1,8b3aa105282895119879c51ec0d7173bfd01e81a22c9e4...,2024-07-29 00:00:34,0x46B3fDF7b5CDe91Ac049936bF0bDb12c5d22202e,0x00000000041d945c46E073F0048cEf510D148dEA,3.67,swap,2.32,1153272.35,3235330.01
2,833370ccc3206456703c1a980c3b913c12830cc0a0581a...,2024-07-29 00:01:09,0x00000000041d945c46E073F0048cEf510D148dEA,0x46B3fDF7b5CDe91Ac049936bF0bDb12c5d22202e,3.78,buy,2.29,1153279.18,3234871.62
3,79e3486ed7d43b8bebe27e7dd62ae04cea3cc119e0f8d3...,2024-07-29 00:01:43,0x639D2a49bdD0CF37614F46a83e4AA49eAFF1e537,0x639D2a49bdD0CF37614F46a83e4AA49eAFF1e537,3.58,swap,2.34,1153290.26,3235170.30
4,a320a4d11803e3829c0f56874e4332feb2646528753ea6...,2024-07-29 00:02:18,0x00000000041d945c46E073F0048cEf510D148dEA,0x00000000fc65059bDdA566b43E17c310B3076e33,3.63,swap,2.31,1153274.36,3235417.75
...,...,...,...,...,...,...,...,...,...
9995,17d071f0d8f066a78e4d56ed895bd04a30a5c1d69fd886...,2024-08-01 23:57:41,0x00000000fc65059bDdA566b43E17c310B3076e33,0xdA07eFFA7aEe8CD85feb894e33Da9163f082D19e,2.55,swap,2.31,1153285.24,3236092.07
9996,a5f180b4e56b70851cc85aa80e623b7820654480711d10...,2024-08-01 23:58:16,0xdA07eFFA7aEe8CD85feb894e33Da9163f082D19e,0x46B3fDF7b5CDe91Ac049936bF0bDb12c5d22202e,2.90,transfer,2.30,1153254.65,3234902.08
9997,5e5ec1b04cca76d7001aea0396c8dcd53837f32a430755...,2024-08-01 23:58:50,0x000000000231c53e9dCbD5Ee410f065FBc170c29,0x000000000231c53e9dCbD5Ee410f065FBc170c29,2.29,swap,2.31,1153294.32,3235937.56
9998,34fa716a537c8ede36b56b0ffc02932da8f50d9dc01e24...,2024-08-01 23:59:25,0xdA07eFFA7aEe8CD85feb894e33Da9163f082D19e,0x00000000fc65059bDdA566b43E17c310B3076e33,2.92,transfer,2.32,1153282.39,3235571.15


In [14]:
generate_df = generate_df[generate_df.txn_maker_address != generate_df.txn_receiver_address]

In [15]:
generate_df['time_stamp'] = pd.to_datetime(generate_df['time_stamp'])

/home/natu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
sample_df = generate_df.sample(frac=0.01, random_state=42)
sample_df['time_stamp'] = sample_df['time_stamp'] + pd.to_timedelta(np.random.randint(0, 1000, size=len(sample_df)), unit='ms')

In [19]:
generate_df = pd.concat([generate_df, sample_df], ignore_index=True)

In [20]:
generate_df.to_parquet('data/generate_sample.parquet', index=False)